In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read the data from the csv file
data0 = pd.read_csv('data0.csv')

# use half of the data to improve the efficiency
data0 = data0.iloc[0:200000, :]

# delete _V0 from the column names
data0.columns = data0.columns.str.replace('_V0', '')

# read the data from the excel file
non_cancer0 = pd.read_excel('non_cancer0.xlsx')

# use half of the data to improve the efficiency
non_cancer0 = non_cancer0.iloc[0:200000, :]

# delete _V0 from the column names
non_cancer0.columns = non_cancer0.columns.str.replace('_V0', '')

# drop the first column
data0.drop(['Unnamed: 0'], axis=1, inplace=True)
non_cancer0.drop(['Unnamed: 0'], axis=1, inplace=True)

# combine data0 and non_cancer0
data = pd.concat([data0, non_cancer0], axis=1)

# drop the rows with missing values in the first 54 columns
data.dropna(axis=0, how='any', subset=data.columns[0:54], inplace=True)

# drop missing values in data0
data0.dropna(axis=0, how='any', subset=data0.columns[0:54], inplace=True)

In [3]:
# define functions to standardize the data
def standardize(x):
    return (x - x.mean()) / x.std()

def standardize_data(data):
    data_std = pd.DataFrame(columns=data.columns)
    for i in range(data.shape[1]):
        data_std.iloc[:, i] = standardize(data.iloc[:, i])
    return data_std

In [4]:
# creeate a new dataframe for PCA and DL
data_pca_dl = data0.copy()

# take log transformation for columns with mean > 1
log_list = []

for i in range(54):
    if data_pca_dl.iloc[:, i].mean() > 1:
        data_pca_dl.iloc[:, i] = np.log(data_pca_dl.iloc[:, i])
        log_list.append(i)

print(log_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 14, 18, 19, 20, 21, 24, 26, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/panda

In [5]:
# copy the data
data_search = data.copy()

# drop the corresponding rows in data_search
data_search.drop(data_pca_dl[data_pca_dl.iloc[:, 0:55].isin([np.inf, -np.inf]).any(1)].index, inplace=True)

# drop the rows with infinite values
data_pca_dl.drop(data_pca_dl[data_pca_dl.iloc[:, 0:55].isin([np.inf, -np.inf]).any(1)].index, inplace=True)

# add (log) to the column names according to the log_list
for i in log_list:
    data_pca_dl.rename(columns={data_pca_dl.columns[i]: data_pca_dl.columns[i] + '(log)'}, inplace=True)

# standardize the data
data_pca_dl = standardize_data(data_pca_dl)

/var/folders/83/985nqkzx2qz9vgyvjg1p3c9w0000gn/T/ipykernel_27304/4181694511.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data_search.drop(data_pca_dl[data_pca_dl.iloc[:, 0:55].isin([np.inf, -np.inf]).any(1)].index, inplace=True)
/var/folders/83/985nqkzx2qz9vgyvjg1p3c9w0000gn/T/ipykernel_27304/4181694511.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data_pca_dl.drop(data_pca_dl[data_pca_dl.iloc[:, 0:55].isin([np.inf, -np.inf]).any(1)].index, inplace=True)
/var/folders/83/985nqkzx2qz9vgyvjg1p3c9w0000gn/T/ipykernel_27304/3368539109.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data_std.iloc[:, i] = standardize(d

In [6]:
# create a new dataframe for classification
data_class_nmf = data0.copy()

# add a new column to the dataframe for labels
data_class_nmf['label'] = 0

# create a new dataframe for classification
data_class_pca_dl = data_pca_dl.copy()

# add a new column to the dataframe for labels
data_class_pca_dl['label'] = 0

In [7]:
# search for value 1074 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1074, 'label'] = 1

# search for value 1074 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1074, 'label'] = 1

### Try classification with different models

In [8]:
# import the necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.model_selection import train_test_split

In [9]:
# set seed for reproducibility
np.random.seed(1709231)

In [10]:
# kNN
knn = KNeighborsClassifier(n_neighbors=5)

In [11]:
# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, y_train, y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# fit the model
knn.fit(X_train, y_train)

# predict the labels of the test set
y_pred = knn.predict(X_test)

# print the accuracy score
print('Accuracy score:', accuracy_score(y_test, y_pred))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy score: 0.9651055712688286


In [12]:
# create the PCA instance
pca = PCA(n_components=0.95) # 0.95 is the minimum number of components that explain 95% of the variance

# fit on data
pca.fit(data_pca_dl)

PCA(n_components=0.95)

In [13]:
# access values and vectors
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

[6.13973096 4.0113006  3.60376035 3.26686149 2.99285937 2.83148508
 2.18110833 2.04689115 1.92222787 1.82814561 1.73066272 1.67466417
 1.58185274 1.54169448 1.39950549 1.28093874 1.2465844  1.00695823
 0.9737357  0.92232841 0.86994164 0.80105629 0.77339326 0.67307118
 0.63268403 0.60490725 0.55102213 0.53065425 0.47600906 0.46482035
 0.42862283 0.41356035]
[0.11369872 0.07428334 0.0667363  0.06049743 0.05542332 0.05243491
 0.04039089 0.03790539 0.03559681 0.03385455 0.03204931 0.0310123
 0.02929357 0.0285499  0.02591677 0.02372109 0.0230849  0.01864737
 0.01803214 0.01708016 0.01611003 0.01483438 0.0143221  0.01246428
 0.01171637 0.01120199 0.01020411 0.00982693 0.00881498 0.00860778
 0.00793746 0.00765852]


In [14]:
# transform data
data0_pca = pca.transform(data_pca_dl)

# check the shape of the data
data0_pca.shape

(152030, 32)

In [15]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# fit the model
knn.fit(X_train, y_train)

# predict the values
y_pred = knn.predict(X_test)

# check the accuracy
print('Accuracy: ', accuracy_score(y_test, y_pred))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy:  0.9654344537262383


In [16]:
# create the NMF instance
nmf = NMF(n_components=5)

# fit on data
nmf.fit(data0)

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


NMF(n_components=5)

In [17]:
# transform data
data0_nmf = nmf.transform(data0)

# check the shape of the data
data0_nmf.shape

(152270, 5)

In [18]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)

# fit the model
knn.fit(X_train, y_train)

# predict the values
y_pred = knn.predict(X_test)

# check the accuracy
print('Accuracy: ', accuracy_score(y_test, y_pred))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy:  0.9658172982202666


In [19]:
# create the DL instance
DL = MiniBatchDictionaryLearning(n_components=100)

# fit on data
DL.fit(data_pca_dl)

MiniBatchDictionaryLearning(n_components=100)

In [20]:
# transform data
data0_dl = DL.transform(data_pca_dl)

# check the shape of the data
data0_dl.shape

(152030, 100)

In [21]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# fit the model
knn.fit(X_train, y_train)

# predict the values
y_pred = knn.predict(X_test)

# check the accuracy
print('Accuracy: ', accuracy_score(y_test, y_pred))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy:  0.9659277774123528


### Create PCA, NMF and DL models

In [22]:
# create the PCA instance
pca = PCA(n_components=0.95) # 0.95 is the minimum number of components that explain 95% of the variance

# fit on data
pca.fit(data_pca_dl)

# access values and vectors
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

# transform data
data0_pca = pca.transform(data_pca_dl)

# create the NMF instance
nmf = NMF(n_components=pca.n_components_)

# fit on data
nmf.fit(data0)

# transform data
data0_nmf = nmf.transform(data0)

# create the DL instance
DL = MiniBatchDictionaryLearning(n_components=100)

# fit on data
DL.fit(data_pca_dl)

# transform data
data0_dl = DL.transform(data_pca_dl)

[6.13973096 4.0113006  3.60376035 3.26686149 2.99285937 2.83148508
 2.18110833 2.04689115 1.92222787 1.82814561 1.73066272 1.67466417
 1.58185274 1.54169448 1.39950549 1.28093874 1.2465844  1.00695823
 0.9737357  0.92232841 0.86994164 0.80105629 0.77339326 0.67307118
 0.63268403 0.60490725 0.55102213 0.53065425 0.47600906 0.46482035
 0.42862283 0.41356035]
[0.11369872 0.07428334 0.0667363  0.06049743 0.05542332 0.05243491
 0.04039089 0.03790539 0.03559681 0.03385455 0.03204931 0.0310123
 0.02929357 0.0285499  0.02591677 0.02372109 0.0230849  0.01864737
 0.01803214 0.01708016 0.01611003 0.01483438 0.0143221  0.01246428
 0.01171637 0.01120199 0.01020411 0.00982693 0.00881498 0.00860778
 0.00793746 0.00765852]


/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


### write scripts for 16 diseases for choice of k

In [23]:
# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1074 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1074['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1074.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1074.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1074.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1074.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [24]:
X_knn_test_accuracy_1074

,k,KNN,PCA,NMF,DL
0,3,0.962376,0.962409,0.963946,0.963231
1,4,0.966059,0.96629,0.966211,0.96652
2,5,0.965106,0.965434,0.965817,0.965928
3,6,0.966158,0.966388,0.966376,0.966783
4,7,0.965698,0.965763,0.966211,0.966388


In [25]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1075 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1075, 'label'] = 1

# search for value 1075 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1075, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1075 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1075['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1075.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1075.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1075.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1075.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [26]:
X_knn_test_accuracy_1075

,k,KNN,PCA,NMF,DL
0,3,0.973953,0.973492,0.975143,0.974446
1,4,0.975893,0.976156,0.976292,0.976156
2,5,0.975564,0.97563,0.976259,0.975926
3,6,0.976255,0.976386,0.976391,0.976353
4,7,0.97609,0.976222,0.976423,0.97632


In [27]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1309 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1309, 'label'] = 1

# search for value 1309 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1309, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1309 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1309['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1309.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1309.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1309.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1309.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [28]:
X_knn_test_accuracy_1309

,k,KNN,PCA,NMF,DL
0,3,0.982997,0.983457,0.982662,0.983128
1,4,0.983951,0.983951,0.983582,0.983951
2,5,0.983951,0.983918,0.983582,0.983951
3,6,0.983983,0.983983,0.983647,0.983983
4,7,0.983951,0.983983,0.983647,0.983983


In [29]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1349 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1349, 'label'] = 1

# search for value 1349 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1349, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1349 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1349['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1349.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1349.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1349.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1349.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [30]:
X_knn_test_accuracy_1349

,k,KNN,PCA,NMF,DL
0,3,0.991811,0.991745,0.991528,0.991679
1,4,0.99214,0.99214,0.991791,0.99214
2,5,0.992074,0.992107,0.991791,0.99214
3,6,0.99214,0.99214,0.991791,0.99214
4,7,0.99214,0.99214,0.991791,0.99214


In [31]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1351 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1351, 'label'] = 1

# search for value 1351 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1351, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1351 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1351['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1351.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1351.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1351.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1351.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [32]:
X_knn_test_accuracy_1351

,k,KNN,PCA,NMF,DL
0,3,0.98349,0.98372,0.984042,0.982997
1,4,0.984707,0.984674,0.985059,0.984641
2,5,0.984674,0.984674,0.985059,0.984641
3,6,0.984707,0.984707,0.985059,0.984707
4,7,0.984707,0.984707,0.985059,0.984674


In [33]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1352 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1352, 'label'] = 1

# search for value 1352 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1352, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1352 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1352['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1352.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1352.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1352.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1352.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [34]:
X_knn_test_accuracy_1352

,k,KNN,PCA,NMF,DL
0,3,0.996744,0.996777,0.996651,0.996777
1,4,0.996843,0.996843,0.996716,0.996843
2,5,0.996843,0.996843,0.996716,0.996843
3,6,0.996843,0.996843,0.996716,0.996843
4,7,0.996843,0.996843,0.996716,0.996843


In [35]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1367 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1367, 'label'] = 1

# search for value 1367 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1367, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1367 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1367['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1367.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1367.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1367.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1367.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [36]:
X_knn_test_accuracy_1367

,k,KNN,PCA,NMF,DL
0,3,0.995363,0.995429,0.995666,0.995396
1,4,0.995494,0.995494,0.995731,0.995494
2,5,0.995494,0.995494,0.995731,0.995494
3,6,0.995494,0.995494,0.995731,0.995494
4,7,0.995494,0.995494,0.995731,0.995494


In [37]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1466 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1466, 'label'] = 1

# search for value 1466 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1466, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1466 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1466['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1466.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1466.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1466.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1466.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [38]:
X_knn_test_accuracy_1466

,k,KNN,PCA,NMF,DL
0,3,0.984016,0.984148,0.985453,0.984247
1,4,0.985069,0.985003,0.986767,0.984806
2,5,0.98497,0.984937,0.98657,0.984806
3,6,0.985036,0.985036,0.986898,0.984937
4,7,0.985003,0.985036,0.986865,0.984937


In [39]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1155 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1155, 'label'] = 1

# search for value 1155 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1155, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1155 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1155['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1155.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1155.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1155.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1155.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [40]:
X_knn_test_accuracy_1155

,k,KNN,PCA,NMF,DL
0,3,0.999046,0.999079,0.998752,0.999079
1,4,0.999079,0.999079,0.998752,0.999079
2,5,0.999079,0.999079,0.998752,0.999079
3,6,0.999079,0.999079,0.998752,0.999079
4,7,0.999079,0.999079,0.998752,0.999079


In [41]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1426 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1426, 'label'] = 1

# search for value 1426 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1426, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1426 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1426['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1426.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1426.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1426.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1426.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [42]:
X_knn_test_accuracy_1426

,k,KNN,PCA,NMF,DL
0,3,0.999934,0.999934,0.999836,0.999934
1,4,0.999934,0.999934,0.999836,0.999934
2,5,0.999934,0.999934,0.999836,0.999934
3,6,0.999934,0.999934,0.999836,0.999934
4,7,0.999934,0.999934,0.999836,0.999934


In [43]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1275 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1275, 'label'] = 1

# search for value 1275 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1275, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1275 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1275['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1275.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1275.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1275.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1275.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [44]:
X_knn_test_accuracy_1275

,k,KNN,PCA,NMF,DL
0,3,0.99852,0.998454,0.998325,0.99852
1,4,0.998553,0.998553,0.998325,0.998553
2,5,0.998553,0.998553,0.998325,0.998553
3,6,0.998553,0.998553,0.998325,0.998553
4,7,0.998553,0.998553,0.998325,0.998553


In [45]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1450 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1450, 'label'] = 1

# search for value 1450 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1450, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1450 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1450['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1450.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1450.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1450.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1450.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [46]:
X_knn_test_accuracy_1450

,k,KNN,PCA,NMF,DL
0,3,0.999934,0.999934,0.999869,0.999934
1,4,0.999934,0.999934,0.999869,0.999934
2,5,0.999934,0.999934,0.999869,0.999934
3,6,0.999934,0.999934,0.999869,0.999934
4,7,0.999934,0.999934,0.999869,0.999934


In [47]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1237 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1237, 'label'] = 1

# search for value 1237 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1237, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1237 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1237['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1237.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1237.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1237.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1237.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [48]:
X_knn_test_accuracy_1237

,k,KNN,PCA,NMF,DL
0,3,0.999901,0.999901,0.999901,0.999901
1,4,0.999901,0.999901,0.999901,0.999901
2,5,0.999901,0.999901,0.999901,0.999901
3,6,0.999901,0.999901,0.999901,0.999901
4,7,0.999901,0.999901,0.999901,0.999901


In [49]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1516 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1516, 'label'] = 1

# search for value 1516 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1516, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1516 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1516['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1516.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1516.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1516.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1516.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [50]:
X_knn_test_accuracy_1516

,k,KNN,PCA,NMF,DL
0,3,0.998487,0.998454,0.998424,0.99852
1,4,0.99852,0.99852,0.998424,0.99852
2,5,0.99852,0.99852,0.998424,0.99852
3,6,0.99852,0.99852,0.998424,0.99852
4,7,0.99852,0.99852,0.998424,0.99852


In [51]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1614 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1614, 'label'] = 1

# search for value 1614 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1614, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1614 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1614['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1614.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1614.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1614.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1614.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [52]:
X_knn_test_accuracy_1614

,k,KNN,PCA,NMF,DL
0,3,0.998816,0.998816,0.99849,0.99875
1,4,0.998816,0.998816,0.99849,0.998816
2,5,0.998816,0.998816,0.99849,0.998816
3,6,0.998816,0.998816,0.99849,0.998816
4,7,0.998816,0.998816,0.99849,0.998816


In [53]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1630 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1630, 'label'] = 1

# search for value 1630 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1630, 'label'] = 1

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1630 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1630['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1630.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1630.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1630.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1630.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [54]:
X_knn_test_accuracy_1630

,k,KNN,PCA,NMF,DL
0,3,0.99977,0.99977,0.999901,0.99977
1,4,0.99977,0.99977,0.999901,0.99977
2,5,0.99977,0.99977,0.999901,0.99977
3,6,0.99977,0.99977,0.999901,0.99977
4,7,0.99977,0.99977,0.999901,0.99977


### for the choice of the dimensionality

In [55]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1074 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1074, 'label'] = 1

# search for value 1074 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1074, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1074 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1074['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1074['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1074.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1074.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [56]:
X_nmf_dl_test_accuracy_1074

,n_components,NMF,number of columns,DL
0,8,0.966376,60,0.966882
1,16,0.96631,80,0.966882
2,24,0.966343,100,0.966586
3,32,0.96631,120,0.966586
4,40,0.966376,140,0.966651


In [57]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1075 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1075, 'label'] = 1

# search for value 1075 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1075, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1075 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1075['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1075['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1075.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1075.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [58]:
X_nmf_dl_test_accuracy_1075

,n_components,NMF,number of columns,DL
0,8,0.976358,60,0.97632
1,16,0.976391,80,0.976288
2,24,0.976423,100,0.976288
3,32,0.976259,120,0.976222
4,40,0.976325,140,0.976288


In [59]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1309 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1309, 'label'] = 1

# search for value 1309 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1309, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1309 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1309['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1309['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1309.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1309.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [60]:
X_nmf_dl_test_accuracy_1309

,n_components,NMF,number of columns,DL
0,8,0.983647,60,0.983983
1,16,0.983647,80,0.983983
2,24,0.983647,100,0.983983
3,32,0.983647,120,0.983983
4,40,0.983647,140,0.983983


In [61]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1349 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1349, 'label'] = 1

# search for value 1349 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1349, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1349 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1349['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1349['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1349.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1349.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [62]:
X_nmf_dl_test_accuracy_1349

,n_components,NMF,number of columns,DL
0,8,0.991791,60,0.99214
1,16,0.991791,80,0.99214
2,24,0.991791,100,0.99214
3,32,0.991791,120,0.99214
4,40,0.991791,140,0.99214


In [63]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1155 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1155, 'label'] = 1

# search for value 1155 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1155, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1155 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1155['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1155['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1155.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1155.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [64]:
X_nmf_dl_test_accuracy_1155

,n_components,NMF,number of columns,DL
0,8,0.998752,60,0.999079
1,16,0.998752,80,0.999079
2,24,0.998752,100,0.999079
3,32,0.998752,120,0.999079
4,40,0.998752,140,0.999079


In [65]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1426 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1426, 'label'] = 1

# search for value 1426 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1426, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1426 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1426['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1426['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1426.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1426.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [66]:
X_nmf_dl_test_accuracy_1426

,n_components,NMF,number of columns,DL
0,8,0.999836,60,0.999934
1,16,0.999836,80,0.999934
2,24,0.999836,100,0.999934
3,32,0.999836,120,0.999934
4,40,0.999836,140,0.999934


In [67]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1275 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1275, 'label'] = 1

# search for value 1275 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1275, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1275 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1275['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1275['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1275.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1275.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [68]:
X_nmf_dl_test_accuracy_1275

,n_components,NMF,number of columns,DL
0,8,0.998325,60,0.998553
1,16,0.998325,80,0.998553
2,24,0.998325,100,0.998553
3,32,0.998325,120,0.998553
4,40,0.998325,140,0.998553


In [69]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# search for value 1450 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1450, 'label'] = 1

# search for value 1450 in the last 34 columns in data and label the corresponding rows in data_class as 1
for i in range(34):
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1450, 'label'] = 1

# create a table to store the accuracy scores for NMF with range of n_components = 8,16,24,32,40 and DL with range of number of columns = 60,80,100,120,140
X_nmf_dl_test_accuracy_1450 = pd.DataFrame(columns=['n_components','NMF','number of columns','DL'])
X_nmf_dl_test_accuracy_1450['n_components'] = [8,16,24,32,40]
X_nmf_dl_test_accuracy_1450['number of columns'] = [60,80,100,120,140]

# Use the first 80% of the data as the training set and the rest as the test set.
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data_class_nmf.iloc[:, 0:54], data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)


# train the model
for i in range(1,6):

    # create the NMF instance
    nmf = NMF(n_components=i*8)

    # fit on data
    nmf.fit(X_train_nmf)

    # transform data
    X_train_nmf = nmf.transform(X_train_nmf)
    X_test_nmf = nmf.transform(X_test_nmf)

    # create the DL instance
    dl = MiniBatchDictionaryLearning(n_components=60+i*20)

    # fit on data
    dl.fit(X_train_dl)

    # transform data
    X_train_dl = dl.transform(X_train_dl)
    X_test_dl = dl.transform(X_test_dl)

    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(X_train_nmf, Y_train_nmf)
    X_nmf_dl_test_accuracy_1450.loc[i-1,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_nmf_dl_test_accuracy_1450.loc[i-1,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  

In [70]:
X_nmf_dl_test_accuracy_1450

,n_components,NMF,number of columns,DL
0,8,0.999869,60,0.999934
1,16,0.999869,80,0.999934
2,24,0.999869,100,0.999934
3,32,0.999869,120,0.999934
4,40,0.999869,140,0.999934


### classification with two diesases

In [71]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# do a classification for disease 1074 and 1075, no disease of 1074 or 1075 labelled as 0, disease 1074 labelled as 1, disease 1075 labelled as 2, both diseases labelled as 3
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1074, 'label'] = 1
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1074, 'label'] = 1

for i in range(34):
    if data_class_nmf.iloc[i,54] == 1:
        data_class_nmf.loc[data.iloc[:, i+54] == 1075, 'label'] = 3
    else:
        data_class_nmf.loc[data.iloc[:, i+54] == 1075, 'label'] = 2

    if data_class_pca_dl.iloc[i,54] == 1:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1075, 'label'] = 3
    else:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1075, 'label'] = 2

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1074_1075 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1074_1075['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1074_1075.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1074_1075.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1074_1075.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1074_1075.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [72]:
X_knn_test_accuracy_1074_1075

,k,KNN,PCA,NMF,DL
0,3,0.949319,0.949089,0.950877,0.950339
1,4,0.951325,0.951753,0.952551,0.952115
2,5,0.952115,0.952213,0.95311,0.952871
3,6,0.952575,0.952805,0.953077,0.953397
4,7,0.95274,0.95297,0.953208,0.953364


In [73]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# none labelled as 0, disease 1309 labelled as 1, disease 1349 labelled as 2, both labelled as 3
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1309, 'label'] = 1
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1309, 'label'] = 1

for i in range(34):
    if data_class_nmf.iloc[i,54] == 1:
        data_class_nmf.loc[data.iloc[:, i+54] == 1349, 'label'] = 3
    else:
        data_class_nmf.loc[data.iloc[:, i+54] == 1349, 'label'] = 2

    if data_class_pca_dl.iloc[i,54] == 1:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1349, 'label'] = 3
    else:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1349, 'label'] = 2

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1309_1349 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1309_1349['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1309_1349.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1309_1349.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1309_1349.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1309_1349.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [74]:
X_knn_test_accuracy_1309_1349

,k,KNN,PCA,NMF,DL
0,3,0.975301,0.975663,0.974486,0.975268
1,4,0.976419,0.976419,0.975603,0.976452
2,5,0.976419,0.976452,0.975668,0.976518
3,6,0.976551,0.976551,0.975701,0.976551
4,7,0.976551,0.976551,0.975734,0.976551


In [75]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# none labelled as 0, disease 1351 labelled as 1, disease 1352 labelled as 2, both labelled as 3
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1351, 'label'] = 1
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1351, 'label'] = 1

for i in range(34):
    if data_class_nmf.iloc[i,54] == 1:
        data_class_nmf.loc[data.iloc[:, i+54] == 1352, 'label'] = 3
    else:
        data_class_nmf.loc[data.iloc[:, i+54] == 1352, 'label'] = 2

    if data_class_pca_dl.iloc[i,54] == 1:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1352, 'label'] = 3
    else:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1352, 'label'] = 2

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1351_1352 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1351_1352['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1351_1352.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1351_1352.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1351_1352.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1351_1352.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [76]:
X_knn_test_accuracy_1351_1352

,k,KNN,PCA,NMF,DL
0,3,0.98053,0.980793,0.980856,0.980201
1,4,0.981714,0.981747,0.981907,0.981714
2,5,0.981747,0.981747,0.98194,0.981714
3,6,0.98178,0.98178,0.98194,0.981747
4,7,0.98178,0.98178,0.98194,0.981747


In [77]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# none labelled as 0, disease 1155 labelled as 1, disease 1426 labelled as 2, both labelled as 3
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1155, 'label'] = 1
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1155, 'label'] = 1

for i in range(34):
    if data_class_nmf.iloc[i,54] == 1:
        data_class_nmf.loc[data.iloc[:, i+54] == 1426, 'label'] = 3
    else:
        data_class_nmf.loc[data.iloc[:, i+54] == 1426, 'label'] = 2

    if data_class_pca_dl.iloc[i,54] == 1:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1426, 'label'] = 3
    else:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1426, 'label'] = 2

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1155_1426 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1155_1426['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1155_1426.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1155_1426.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1155_1426.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1155_1426.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [78]:
X_knn_test_accuracy_1155_1426

,k,KNN,PCA,NMF,DL
0,3,0.99898,0.999013,0.998588,0.999013
1,4,0.999013,0.999013,0.998588,0.999013
2,5,0.999013,0.999013,0.998588,0.999013
3,6,0.999013,0.999013,0.998588,0.999013
4,7,0.999013,0.999013,0.998588,0.999013


In [79]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# none labelled as 0, disease 1275 labelled as 1, disease 1450 labelled as 2, both labelled as 3
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1275, 'label'] = 1
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1275, 'label'] = 1

for i in range(34):
    if data_class_nmf.iloc[i,54] == 1:
        data_class_nmf.loc[data.iloc[:, i+54] == 1450, 'label'] = 3
    else:
        data_class_nmf.loc[data.iloc[:, i+54] == 1450, 'label'] = 2

    if data_class_pca_dl.iloc[i,54] == 1:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1450, 'label'] = 3
    else:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1450, 'label'] = 2

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1275_1450 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1275_1450['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1275_1450.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1275_1450.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1275_1450.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1275_1450.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [80]:
X_knn_test_accuracy_1275_1450

,k,KNN,PCA,NMF,DL
0,3,0.998454,0.998388,0.998194,0.998454
1,4,0.998487,0.998487,0.998194,0.998487
2,5,0.998487,0.998487,0.998194,0.998487
3,6,0.998487,0.998487,0.998194,0.998487
4,7,0.998487,0.998487,0.998194,0.998487


In [81]:
data_class_nmf['label'] = 0
data_class_pca_dl['label'] = 0

# none labelled as 0, disease 1237 labelled as 1, disease 1516 labelled as 2, both labelled as 3
for i in range(34):
    data_class_nmf.loc[data.iloc[:, i+54] == 1237, 'label'] = 1
    data_class_pca_dl.loc[data.iloc[:, i+54] == 1237, 'label'] = 1

for i in range(34):
    if data_class_nmf.iloc[i,54] == 1:
        data_class_nmf.loc[data.iloc[:, i+54] == 1516, 'label'] = 3
    else:
        data_class_nmf.loc[data.iloc[:, i+54] == 1516, 'label'] = 2

    if data_class_pca_dl.iloc[i,54] == 1:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1516, 'label'] = 3
    else:
        data_class_pca_dl.loc[data.iloc[:, i+54] == 1516, 'label'] = 2

# create a table to store the accuracy scores for different methods with k of KNN ranging from 3 to 7
X_knn_test_accuracy_1237_1516 = pd.DataFrame(columns=['k','KNN','PCA','NMF','DL'])
X_knn_test_accuracy_1237_1516['k'] = range(3,8)

# Use the first 80% of the data as the training set and the rest as the test set.
X_train, X_test, Y_train, Y_test = train_test_split(data_class_pca_dl.iloc[:, 0:54], data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_pca, X_test_pca, Y_train_pca, Y_test_pca = train_test_split(data0_pca, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)
X_train_nmf, X_test_nmf, Y_train_nmf, Y_test_nmf = train_test_split(data0_nmf, data_class_nmf.iloc[:, 54], test_size=0.2, random_state=42)
X_train_dl, X_test_dl, Y_train_dl, Y_test_dl = train_test_split(data0_dl, data_class_pca_dl.iloc[:, 54], test_size=0.2, random_state=42)

# train the model
for i in range(3,8):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, Y_train)
    X_knn_test_accuracy_1237_1516.loc[i-3,'KNN'] = accuracy_score(Y_test, knn.predict(X_test))
    knn.fit(X_train_pca, Y_train_pca)
    X_knn_test_accuracy_1237_1516.loc[i-3,'PCA'] = accuracy_score(Y_test_pca, knn.predict(X_test_pca))
    knn.fit(X_train_nmf, Y_train_nmf)
    X_knn_test_accuracy_1237_1516.loc[i-3,'NMF'] = accuracy_score(Y_test_nmf, knn.predict(X_test_nmf))
    knn.fit(X_train_dl, Y_train_dl)
    X_knn_test_accuracy_1237_1516.loc[i-3,'DL'] = accuracy_score(Y_test_dl, knn.predict(X_test_dl))

/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/xt319/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer b

In [82]:
X_knn_test_accuracy_1237_1516

,k,KNN,PCA,NMF,DL
0,3,0.998388,0.998356,0.998325,0.998421
1,4,0.998421,0.998421,0.998325,0.998421
2,5,0.998421,0.998421,0.998325,0.998421
3,6,0.998421,0.998421,0.998325,0.998421
4,7,0.998421,0.998421,0.998325,0.998421
